In [50]:
import pandas as pd
import psycopg2

host = "localhost"
database = "riskdb"
user = "postgres"
password = "12345"

connection = psycopg2.connect(host=host, database=database, user=user, password=password)
cursor = connection.cursor()
cursor.execute("SELECT * FROM dataset")
data_tr = cursor.fetchall()
cursor.execute("SELECT * FROM test")
data_test = cursor.fetchall()
cursor.close()
connection.close()

train = pd.DataFrame(data_tr)
test = pd.DataFrame(data_test)

In [51]:
print(test)

                                                    0      1
0   Цена настоящего договора 73 500 (Семьдесят три...   Цена
1   Цена Договора является твердой и определяется ...   Цена
2   Цена договора включает в себя стоимость Товара...   Цена
3                              Аванс не предусмотрен.  Аванс
4   Сумма по Договору, подлежащая уплате Поставщик...  Налог
5   Цена Договора может быть изменена, если по пре...   Цена
6   При этом по соглашению Сторон допускается изме...   Цена
7   Поставщик осуществляет передачу в течение 10 (...   Срок
8   Пеня начисляется за каждый день просрочки испо...   Пени
9    За каждый факт неисполнения или ненадлежащего...  Штраф
10   За каждый факт неисполнения Заказчиком обязат...  Штраф
11  Товар поставляется согласно техническому задан...   Срок


In [52]:
print(train)

                                                    0       1
0   Пеня начисляется за каждый день просрочки испо...    Пени
1   10 процентов цены контракта (этапа) в случае, ...   Штраф
2   5 процентов цены контракта (этапа) в случае, е...   Штраф
3   1 процент цены контракта (этапа) в случае, есл...   Штраф
4   0,5 процента цены контракта (этапа) в случае, ...   Штраф
..                                                ...     ...
84  В рамках экспертизы поставленного Товара на со...  Другое
85  Заказчик вправе для проведения экспертизы Това...  Другое
86  По результатам проведенной экспертизы Товара, ...  Другое
87  Заказчик вправе не отказывать в приемке постав...  Другое
88  В случае привлечения Заказчиком для проведения...  Другое

[89 rows x 2 columns]


In [53]:
train = train.rename(columns={0: 'paragraf', 1: "target"})
test = test.rename(columns={0: 'paragraf', 1: "target"})

In [54]:
print(train)

                                             paragraf  target
0   Пеня начисляется за каждый день просрочки испо...    Пени
1   10 процентов цены контракта (этапа) в случае, ...   Штраф
2   5 процентов цены контракта (этапа) в случае, е...   Штраф
3   1 процент цены контракта (этапа) в случае, есл...   Штраф
4   0,5 процента цены контракта (этапа) в случае, ...   Штраф
..                                                ...     ...
84  В рамках экспертизы поставленного Товара на со...  Другое
85  Заказчик вправе для проведения экспертизы Това...  Другое
86  По результатам проведенной экспертизы Товара, ...  Другое
87  Заказчик вправе не отказывать в приемке постав...  Другое
88  В случае привлечения Заказчиком для проведения...  Другое

[89 rows x 2 columns]


In [55]:
print(test)

                                             paragraf target
0   Цена настоящего договора 73 500 (Семьдесят три...   Цена
1   Цена Договора является твердой и определяется ...   Цена
2   Цена договора включает в себя стоимость Товара...   Цена
3                              Аванс не предусмотрен.  Аванс
4   Сумма по Договору, подлежащая уплате Поставщик...  Налог
5   Цена Договора может быть изменена, если по пре...   Цена
6   При этом по соглашению Сторон допускается изме...   Цена
7   Поставщик осуществляет передачу в течение 10 (...   Срок
8   Пеня начисляется за каждый день просрочки испо...   Пени
9    За каждый факт неисполнения или ненадлежащего...  Штраф
10   За каждый факт неисполнения Заказчиком обязат...  Штраф
11  Товар поставляется согласно техническому задан...   Срок


In [56]:
classes = {"Пени": 0, 
           "Штраф": 1, 
           "Просрочка": 2, 
           "Цена": 3, 
           "Аванс": 4, 
           "Налог": 5, 
           "Срок": 7,
           "Сроки": 7,
           "Изменение цены": 3,
           "Другое": 8}

In [57]:
X_list_train = train['paragraf'].tolist()
Y_list_train = train['target'].tolist()
X_list_test = test['paragraf'].tolist()
Y_list_test = test['target'].tolist()
for i in range(len(Y_list_test)):
    Y_list_test[i] = classes[Y_list_test[i]]
for i in range(len(Y_list_train)):
    Y_list_train[i] = classes[Y_list_train[i]]

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_list_train)
classifier = LinearSVC()
classifier.fit(X_train_tfidf, Y_list_train)
X_test_tfidf = vectorizer.transform(X_list_test)
predictions = classifier.predict(X_test_tfidf)
accuracy = 0
for doc, label, prediction in zip(X_list_test, Y_list_test, predictions):
    print('Document:', doc)
    print('True Label:', label)
    print('Predicted Label:', prediction)
    print('---')
    if label == prediction:
        accuracy += 1
accuracy /= 11
print(accuracy)

Document: Цена настоящего договора 73 500 (Семьдесят три тысячи пятьсот) рублей, 00 коп. без налога (НДС).
True Label: 3
Predicted Label: 3
---
Document: Цена Договора является твердой и определяется на весь срок исполнения Договора, за исключением случаев, установленных в Законе о контрактной системе
True Label: 3
Predicted Label: 3
---
Document: Цена договора включает в себя стоимость Товара, расходы на выполнение Спецификации (Приложение №1) в полном объеме, перевозку
True Label: 3
Predicted Label: 8
---
Document: Аванс не предусмотрен.
True Label: 4
Predicted Label: 8
---
Document: Сумма по Договору, подлежащая уплате Поставщику, уменьшается на размер налогов, сборов и иных обязательных платежей в бюджеты бюджетной системы Российской Федерации, связанных с оплатой Договора, если в соответствии с законодательством Российской Федерации о налогах и сборах такие налоги, сборы и иные обязательные платежи подлежат уплате в бюджеты бюджетной системы Российской Федерации Заказчиком.
True L

C:\Users\senio\PycharmProjects\TenderHackRiskAnalysis\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
